# [추가] case 3 : 제지공정 장애 예측 모델 평가



## 1.환경준비

### 1) 라이브러리 로딩

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.ensemble import IsolationForest # Isolation Forest!

### 2) 데이터셋 불러오기

![](https://keralakaumudi.com/web-news/en/2020/04/NMAN0141956/image/paper-mill.1.582102.jpg)

In [ ]:
# 공정 데이터 불러오기
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/processminer2.csv"
data = pd.read_csv(path)
data['time'] = pd.to_datetime(data['time'], format = '%m/%d/%y %H:%M')
data2 = data.loc[data['time'].between('1999-05-01', '1999-05-13')]
data2.head()

### 3) 필요 함수들 생성

#### ① anomaly_score_plot



In [ ]:
def anomaly_score_plot(y, score, threshold=0):
    error_df = pd.DataFrame({'anomaly_score': score, 'target': y})
    error_df = error_df.reset_index()

    groups = error_df.groupby('target')
    fig, ax = plt.subplots()
    for name, group in groups:
        ax.plot(group.index, group.anomaly_score, marker='o', ms=3.5, linestyle='',
                label= "Abnormal" if name == 1 else "Normal")
    ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
    ax.legend()
    plt.title("anomaly_score for different classes")
    plt.ylabel("anomaly_score")
    plt.xlabel("Data point index")
    plt.show()

    return error_df

#### ② precision, recall, f1 curve

> * sklearn에서는 precision, recall curve만 제공됩니다. 
* 그래서, f1 curve도 추가해서 구하고, plot을 그립니다.



In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_rec_f1_curve(y, score, pos = 1) :
    precision, recall, thresholds  = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    plt.plot(thresholds, np.delete(precision, -1), label = 'precision')
    plt.plot(thresholds, np.delete(recall, -1), label = 'recall')
    plt.plot(thresholds, np.delete(f1, -1), label = 'f1')
    plt.xlabel('Anomaly Score')
    plt.legend()
    plt.grid()
    plt.show()

    return precision, recall, f1, thresholds

#### ③ threshold로 잘랐을 때, 분류 평가 함수


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_report2(y, pred, thresholds):
    pred_temp = np.where(pred > thresholds , 1, 0)

    print('< confusion matrix >\n')
    print(confusion_matrix(y, pred_temp))
    print('\n' + '='*60 + '\n')

    print('< classification_report >\n')
    print(classification_report(y, pred_temp))

    return confusion_matrix(y, pred_temp)

## 2.데이터 준비

### 1) 불필요한 변수 제거

불필요한 변수 제거 : time

In [ ]:
data2 = data2.drop('time', axis=1)

In [ ]:
data2.shape

### 2) 데이터 분할

* x, y 분할

In [ ]:
target = 'y'
x = data2.drop(target, axis = 1)
y = data2.loc[:, target]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 3000, random_state = 2022)

## 3.모델링

In [ ]:
y_train.value_counts()/y_train.shape[0]

In [ ]:
model = IsolationForest(contamination=0.013, random_state=2022)
model.fit(x_train)

In [ ]:
# 학습 데이터에 대한 abnormal 점수 계산
score = model.score_samples(x_train)
score = -1 * score

sns.histplot(score, kde = True)
plt.show()

* 예측

In [ ]:
score = model.score_samples(x_val)
score = -1 * score

plt.figure(figsize = (12,8))
precision, recall, f1, thresholds = prec_rec_f1_curve(y_val, score)

In [ ]:
result = anomaly_score_plot(y_val, score, threshold=0.5)

In [ ]:
f1_max_thr = thresholds[np.argmax(f1)]

f1_max_thr

In [ ]:
classification_report2(y_val, score, f1_max_thr)

## 4.비즈니스 관점에서의 모델 평가

> * 한 롤로 종이를 말다가 찢어지는 사고가 하루에 한번 정도 발생. 
* 이때마다 공정 중단 및 수율 저하 등, 평균적으로 100백만원의 손실이 발생
* 이를 사전에 감지하는 것은 굉장히 어려움. 이런 사고를 5%만 감소시키더라도 회사 입장에서는 상당한 비용 절감효과 예상.
* 장애가 예상된다면, 속도를 줄여 장애를 예방할 수 있다. 단, 속도를 줄이면 생산성이 저하되므로, 1회당 평균 3만원의 손실이 발생됩니다.


### 1) 비즈니스 기대가치 매트릭스

In [ ]:
bv = np.array([[0,3],[100,3]])
bv

### 2) threshold 값을 조정하면서 cost를 계산해 봅시다.

#### ① 비교대상(Base Cost) 계산

* 1) 예방활동을 하지 않고 장애 발생에 대해 조치할 경우, 비용 계산 ==> 계산가능
* 2) 수행 중인 예방활동 비용 + 장애발생 비용 계산 ==> 이 부분은 알 수 없으므로 여기서는 다루지 않음

In [ ]:
base_cost = result['target'].sum() * 100
base_cost

#### ② threshold 값 조정하며 Cost 계산

In [ ]:
cost = []
thresholds = np.linspace(result['anomaly_score'].min(), result['anomaly_score'].max(), 200)

for v in thresholds :
    y_pred = np.where(result['anomaly_score']> v, 1, 0)
    cm = confusion_matrix(result['target'], y_pred)
    cost.append(np.sum(cm * bv))

cost = np.array(cost)

### 3) 비교 그래프를 그려 봅시다.

In [ ]:
min_tres = round(thresholds[np.argmin(cost)],5)

print(f'Anomaly Score : {min_tres},  Min Cost : {min(cost)}' )
plt.figure(figsize = (15, 8))
plt.plot(thresholds, cost)

plt.axhline(base_cost, color = 'r', linestyle = ":")
plt.axhline(min(cost), color = 'g', linestyle = ":")
plt.axvline(min_tres, color = 'g', linestyle = ":")

plt.text(0.4, base_cost+30, f"base cost : {base_cost}", color = 'r')
plt.text(0.4, min(cost)+30, f"min cost : {min(cost)}", color = 'g')

plt.ylim(2500, 4000)
plt.xlabel("Anomaly Score")
plt.ylabel("Cost")
plt.grid()
plt.show()